In [88]:
import pandas as pd
import json
from pandas.io.json import json_normalize #package for flattening json in pandas df
from pathlib import Path
import os
import urllib

import warnings
warnings.filterwarnings("ignore")

### Scrapping de datos de la página web

In [40]:
import requests
from bs4 import BeautifulSoup

URL = 'https://datosabiertos.compraspublicas.gob.ec/OCDS/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

Obtengo nombres de los documentos en la página web

In [41]:
python_jobs = soup.find_all('a')
documentos = []
for p_job in python_jobs:
    link = p_job.find('download')
    texto = p_job.text.split()[1].strip()    
    documentos.append(texto)
# print(documentos)

La siguiente celda puede tardar varios minutos

In [43]:
url_archivos = 'https://datosabiertos.compraspublicas.gob.ec/OCDS/archivos/'
for documento in documentos:
    urllib.request.urlretrieve(url_archivos+documento, '../base_datos/'+documento)
    print('Decargado:', documento)

Decargado: ocds-2020-06-09.json
Decargado: ocds-2020-06-15.json
Decargado: ocds-2020-06-22.json
Decargado: ocds-2020-06-29.json
Decargado: ocds-2020-07-06.json
Decargado: ocds-2020-07-13.json
Decargado: ocds-2020-07-20.json
Decargado: ocds-2020-07-27.json
Decargado: ocds-2020-08-03.json
Decargado: ocds-2020-08-10.json
Decargado: ocds-2020-08-17.json
Decargado: ocds-2020-08-24.json
Decargado: ocds-2020-08-31.json
Decargado: ocds-2020-09-07.json
Decargado: ocds-2020-09-14.json
Decargado: ocds-2020-09-21.json
Decargado: ocds-2020-09-28.json
Decargado: ocds-2020-10-05.json
Decargado: ocds-2020-10-12.json


### Definimos funciones

In [113]:
def json_to_excel(nombre_archivo):      
    p = Path(nombre_archivo)
    with p.open('r', encoding='utf-8') as f:
        data = json.loads(f.read())

    # Aquí se aplana el json nested     
    data_1 = json_normalize(data['releases'])    
    change = data_1.to_json(orient="records")
    parsed = json.loads(change)
    data_aw = json_normalize(parsed,  record_path='awards', record_prefix='awards_', meta = 'id')
    metax = list(data_aw.columns)
    metax.remove('awards_suppliers')
    change = data_aw.to_json(orient="records")
    parsed1 = json.loads(change)
    data_aw1 = json_normalize(parsed1,  record_path='awards_suppliers', record_prefix='awards_suppliers_', meta = metax)
    data_co = json_normalize(parsed,  record_path='contracts', record_prefix='contracts_', meta = 'id')
    data_pa = json_normalize(parsed,  record_path='parties', record_prefix='parties_', meta = 'id')
    columnas_buyer = []
    columnas_supplier = []
    culumnas_pa = data_pa.columns
    for i in culumnas_pa:
        columnas_buyer.append('buyer_'+i)
        columnas_supplier.append('supplier_'+i)
    val_buyer = [data_pa.loc[i].values for i in range(data_pa.shape[0]) if i%2 == 0]
    val_supplier = [data_pa.loc[i].values for i in range(data_pa.shape[0]) if i%2 != 0]
    df_buyer = pd.DataFrame(val_buyer, columns = columnas_buyer) 
    df_supplier = pd.DataFrame(val_supplier, columns = columnas_supplier) 
    df_by_sup = pd.concat([df_buyer, df_supplier], axis=1)
    df_by_sup.drop(columns=['buyer_id'], inplace = True)
    df_by_sup.rename(columns={'supplier_id':'id'}, inplace = True)
    df_final_1 = pd.merge(data_1, data_aw1, how='inner', on='id')
    df_final_2 = pd.merge(df_final_1, data_co, how='inner', on='id')
    df_final_3 = pd.merge(df_final_2, df_by_sup, how='inner', on='id')
    df_final_3.drop(columns=['contracts','parties','awards'], inplace=True)
        
    # Desgloce de items en otro Excel
    change2 = data_co.to_json(orient="records")
    parsed2 = json.loads(change2)    
    df_contract_id = json_normalize(parsed2,  record_path='contracts_items', record_prefix='contracts_items_', meta = ['contracts_id','id'])
    # Reordenar nombre columnas
    id_values = df_contract_id['id']
    df_contract_id.drop(columns=['id'], inplace = True)
    df_contract_id.insert(0,'id',id_values)
#     df_contract_id.head(3)
    
    return df_final_3, df_contract_id

### Cargo archivos Json

In [116]:
path_jsons = '../base_datos/'
path_salida = path_jsons+'convertidos_xls/'

lista_archivos_json = []
for i in os.listdir(path_jsons):
    path = os.path.join(path_jsons, i)
    if not os.path.isdir(path):
        lista_archivos_json.append(i)        
print(lista_archivos_json)

['ocds-2020-06-09.json', 'ocds-2020-06-15.json', 'ocds-2020-06-22.json', 'ocds-2020-06-29.json', 'ocds-2020-07-06.json', 'ocds-2020-07-13.json', 'ocds-2020-07-20.json', 'ocds-2020-07-27.json', 'ocds-2020-08-03.json', 'ocds-2020-08-10.json', 'ocds-2020-08-17.json', 'ocds-2020-08-24.json', 'ocds-2020-08-31.json', 'ocds-2020-09-07.json', 'ocds-2020-09-14.json', 'ocds-2020-09-21.json', 'ocds-2020-09-28.json', 'ocds-2020-10-05.json', 'ocds-2020-10-12.json']


In [131]:
lista_archivos_json[:3]

['ocds-2020-06-09.json', 'ocds-2020-06-15.json', 'ocds-2020-06-22.json']

### Unimos archivos Json Nested

In [138]:
contador = 0
dataframe_anterior = pd.DataFrame()
general = pd.DataFrame()
dataframe_anterior_desglose = pd.DataFrame()
general_desglose = pd.DataFrame()
for archivo in lista_archivos_json[:]:    
    # Excel del json
    data_processed, data_desglose = json_to_excel(path_jsons+archivo)
    print('Procesando:',archivo,'Dimesiones:',data_processed.shape)
    data_processed.to_excel(path_salida+archivo[:-5]+'.xlsx')
    data_processed.to_excel(path_salida+archivo[:-5]+'.xlsx')
    # Excel del desglose de items  
    data_desglose_1 = pd.merge(data_desglose, data_processed_1[['awards_value.amount','id']], how='inner', on='id')
    data_desglose_1.to_excel(path_salida+archivo[:-5]+'_desglose_productos.xlsx')
    
    # Añadir al general
    if contador > 0:
        general = pd.concat([dataframe_anterior, data_processed], ignore_index=True)
        dataframe_anterior = general
        
        general_desglose = pd.concat([dataframe_anterior_desglose, data_desglose_1])
        dataframe_anterior_desglose = general_desglose
    else:
        dataframe_anterior = data_processed.head(1) 
        general = pd.concat([dataframe_anterior, data_processed], ignore_index=True)
        dataframe_anterior = general
        
        dataframe_anterior_desglose = data_desglose_1.head(1) 
        general_desglose = pd.concat([dataframe_anterior_desglose, data_desglose_1])
        dataframe_anterior_desglose = general_desglose
    contador += 1                               

Procesando: ocds-2020-06-09.json Dimesiones: (5484, 55)
Procesando: ocds-2020-06-15.json Dimesiones: (5842, 55)
Procesando: ocds-2020-06-22.json Dimesiones: (6314, 55)
Procesando: ocds-2020-06-29.json Dimesiones: (6582, 55)
Procesando: ocds-2020-07-06.json Dimesiones: (6763, 55)
Procesando: ocds-2020-07-13.json Dimesiones: (6914, 55)
Procesando: ocds-2020-07-20.json Dimesiones: (7081, 55)
Procesando: ocds-2020-07-27.json Dimesiones: (7182, 55)
Procesando: ocds-2020-08-03.json Dimesiones: (7310, 55)
Procesando: ocds-2020-08-10.json Dimesiones: (7386, 55)
Procesando: ocds-2020-08-17.json Dimesiones: (7529, 55)
Procesando: ocds-2020-08-24.json Dimesiones: (7639, 55)
Procesando: ocds-2020-08-31.json Dimesiones: (7703, 55)
Procesando: ocds-2020-09-07.json Dimesiones: (7850, 55)
Procesando: ocds-2020-09-14.json Dimesiones: (8004, 55)
Procesando: ocds-2020-09-21.json Dimesiones: (8055, 55)
Procesando: ocds-2020-09-28.json Dimesiones: (8104, 55)
Procesando: ocds-2020-10-05.json Dimesiones: (81

In [151]:
general_final = general.iloc[1:]
general_desglose_final = general_desglose.iloc[1:]

In [152]:
general_final.shape

(138054, 55)

In [153]:
general_desglose_final.shape

(274719, 9)

### Conversion a CSV

In [167]:
general_final.to_csv(path_salida+'general.csv', index=False)
general_desglose_final.to_csv(path_salida+'general_desglose_productos.csv', index=False)

### Conversion a XLS

In [ ]:
general_final.to_excel(path_salida+'general.xlsx')
general_desglose_final.to_excel(path_salida+'general_desglose_productos.xlsx')